In [14]:
import pandas as pd
df_movie = pd.read_pickle("D:/Project/ATiAI-speakeasy/data/df_movie.pkl")
df_movie['id'] = df_movie['uri'].apply(lambda x: x.split('/')[-1])
list_movies = df_movie['id'].tolist()
df =  pd.read_csv('D:/Project/ATiAI-speakeasy/dataset/14_graph.tsv', sep='\t', header=None)
df.columns = ['subject', 'predicate', 'object']
df['pid'] = df['predicate'].apply(lambda x: x.split('/')[-1])
df['pid'] = df['pid'].apply(lambda x: x.strip('>'))
df['oid'] = df['object'].apply(lambda x: x.split('/')[-1])
df['oid'] = df['oid'].apply(lambda x: x.strip('>'))
df['sid'] = df['subject'].apply(lambda x: x.split('/')[-1])
df['sid'] = df['sid'].apply(lambda x: x.strip('>'))
df['po'] = df.apply(lambda x: (x['pid'], x['oid']), axis=1)
df = df[['sid', 'po']]
df = df.groupby('sid')['po'].apply(list).reset_index()
df = df[df['sid'].isin(list_movies)].reset_index(drop=True)
df.to_pickle("D:/Project/ATiAI-speakeasy/data/df_movie_spo.pkl")

In [15]:
df = pd.read_pickle("D:/Project/ATiAI-speakeasy/data/df_movie_spo.pkl")

In [132]:
# knn
import csv
import numpy as np
import rdflib
from rdflib import RDFS, Graph

graph = Graph().parse('D:/Project/ATiAI-speakeasy/data/14_graph.nt', format='turtle')
# self.graph.parse('data/14_graph.nt', format='turtle')
ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

entity_emb = np.load('D:/Project/ATiAI-speakeasy/data/ddis-graph-embeddings/entity_embeds.npy')
relation_emb = np.load('D:/Project/ATiAI-speakeasy/data/ddis-graph-embeddings/relation_embeds.npy')

with open('D:/Project/ATiAI-speakeasy/data/ddis-graph-embeddings/entity_ids.del', 'r') as ifile:
    ent2id = {ent.split('/')[-1]: int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}

df_movie = pd.read_pickle("D:/Project/ATiAI-speakeasy/data/df_movie.pkl")
df_movie['id'] = df_movie['uri'].apply(lambda x: x.split('/')[-1])
list_movies = df_movie['id'].tolist()
df_movie['emb'] = df_movie['id'].apply(lambda x: entity_emb[ent2id[x]])
df_movie.to_pickle("D:/Project/ATiAI-speakeasy/data/df_movie_emb.pkl")


In [127]:
from sklearn.neighbors import NearestNeighbors
from collections import Counter

topk=250
movie_ids = ['Q1393651', 'Q2036869']
X = df_movie['emb'].tolist()
nbrs = NearestNeighbors(n_neighbors=topk, algorithm='auto').fit(X)
reference_points = [df_movie[df_movie['id']==movie]['emb'].item().tolist() for movie in movie_ids]
rec_ids = []
for p in reference_points:
    _, indices = nbrs.kneighbors([p])
    rec_ids += indices[0].tolist()
    # print(indices)
top_recs = Counter(rec_ids).most_common(topk)

# distances, indices = nbrs.kneighbors(reference_points)
# list_of_movies = df_movie['sid'].values 
# rec_ids = [list_of_movies[i] for i in indices[0]]
# rec_ids = [rec for rec in rec_ids if rec not in movies]

In [128]:
top_recs = Counter(rec_ids).most_common(topk)
top_recs

[(20040, 2),
 (22770, 2),
 (295, 2),
 (19983, 2),
 (18283, 2),
 (17132, 2),
 (17192, 2),
 (18318, 2),
 (23471, 2),
 (17973, 2),
 (20319, 2),
 (3625, 2),
 (13209, 2),
 (2142, 1),
 (20196, 1),
 (20198, 1),
 (23110, 1),
 (4171, 1),
 (9619, 1),
 (2657, 1),
 (8832, 1),
 (5103, 1),
 (16245, 1),
 (9344, 1),
 (867, 1),
 (18478, 1),
 (9353, 1),
 (16777, 1),
 (20848, 1),
 (1838, 1),
 (6154, 1),
 (17397, 1),
 (15698, 1),
 (17109, 1),
 (22775, 1),
 (13008, 1),
 (22160, 1),
 (21708, 1),
 (24159, 1),
 (13572, 1),
 (21586, 1),
 (3096, 1),
 (21397, 1),
 (1195, 1),
 (6425, 1),
 (6870, 1),
 (20945, 1),
 (12911, 1),
 (19662, 1),
 (21336, 1),
 (4657, 1),
 (16650, 1),
 (7700, 1),
 (3673, 1),
 (7870, 1),
 (13096, 1),
 (13745, 1),
 (13001, 1),
 (13995, 1),
 (23185, 1),
 (2581, 1),
 (20616, 1),
 (24177, 1),
 (20086, 1),
 (17370, 1),
 (2559, 1),
 (699, 1),
 (21432, 1),
 (21575, 1),
 (4603, 1),
 (13538, 1),
 (16299, 1),
 (14696, 1),
 (710, 1),
 (15069, 1),
 (22878, 1),
 (21380, 1),
 (19553, 1),
 (6539, 1),
 (13

In [133]:
pd.read_pickle("D:/Project/ATiAI-speakeasy/data/df_movie_emb.pkl")

,uri,label,id,emb
0,http://www.wikidata.org/entity/Q1000825,Jan Dara,Q1000825,"[-186.43588, -186.0038, -54.57119, 74.79744, -..."
1,http://www.wikidata.org/entity/Q1001777,Moondram Pirai,Q1001777,"[-25.526804, -209.46474, 240.5892, 201.7048, -..."
2,http://www.wikidata.org/entity/Q1001943,"Buffalo Bill and the Indians, or Sitting Bull'...",Q1001943,"[-220.76288, -60.65309, -143.36342, 209.50555,..."
3,http://www.wikidata.org/entity/Q100232971,What We Wanted,Q100232971,"[-138.72667, -176.05641, 17.257284, 209.66061,..."
4,http://www.wikidata.org/entity/Q1002480,Wanted: Dead or Alive,Q1002480,"[-180.09285, -175.36699, -95.505936, 235.52911..."
...,...,...,...,...
24379,http://www.wikidata.org/entity/Q23010088,Spider-Man: Homecoming,Q23010088,"[-129.6324, -236.38158, -60.826782, 243.55312,..."
24380,http://www.wikidata.org/entity/Q17738,Star Wars: Episode IV – A New Hope,Q17738,"[-97.96905, -186.11058, -100.17005, 78.66787, ..."
24381,http://www.wikidata.org/entity/Q23781155,Avengers: Endgame,Q23781155,"[-103.17614, -152.22154, 95.77184, 216.63011, ..."
24382,http://www.wikidata.org/entity/Q19590955,Rogue One,Q19590955,"[-111.4378, -18.665018, 11.809444, 196.4732, -..."


In [138]:
df_movie['label'] = df_movie['label'].str.lower()
df_movie[df_movie['label'].str.contains('pocahontas')]

,uri,label,id,emb


In [ ]:
for i in top_recs:
    print(df_movie.iloc[i[0]]['label'])

The Tempest
Quiz Show
Swing Kids
The Other Boleyn Girl
A Room with a View
Sibling Rivalry
Dying Young
Love's Labour's Lost
Sense and Sensibility
The Golden Bowl
Much Ado About Nothing
Jefferson in Paris
Dancing at Lughnasa
Hamlet
Claire Dolan
Make Your Move
The Wrestler
A Perfect Man
Alive
O
The Bling Ring
Broken Flowers
Chelsea Walls
Teknolust
Cradle Will Rock
Moonrise Kingdom
Walking and Talking
The Life Aquatic with Steve Zissou
Michael Clayton
Before the Devil Knows You're Dead
The Phenom
Gods Behaving Badly
The Hurricane
The House of Mirth
Great Expectations
Snow Falling on Cedars
Charlotte's Web
The Ice Storm
The Bourne Identity
Wendigo
Rent
Trainwreck: My Life as an Idiot
Breakfast of Champions
Angela's Ashes
The Accidental Husband
Clockwatchers
The Pelican Brief
Until the End of the World
The Baby-Sitters Club
Che Part 1: The Argentine
The Night Listener
City of Ember
Animal Factory
Picking Up the Pieces
Barry
The Life of David Gale
Find Me Guilty
My Week with Marilyn
Inhale
Br

In [ ]:
for rec in top_recs:
    id = rec[0]
    

In [146]:
for id in ['Q820811','Q15733440','Q1512857','Q1301177','Q606467','Q122713']:
    print(df_movie[df_movie['id']==id]['label'])
# df_movie[df_movie['id'] in ]

15632    final destination 3
Name: label, dtype: object
23612    annabelle
Name: label, dtype: object
17474    the unborn
Name: label, dtype: object
1727    prom night
Name: label, dtype: object
20523    darkness falls
Name: label, dtype: object
21754    insidious
Name: label, dtype: object
